In [1]:
import numpy as np
import pandas as pd
import faiss
import json
from sentence_transformers import SentenceTransformer
# Load model only for queries
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
# Load precomputed embeddings and job IDs
job_embeddings = np.load("job_embeddings.npy")
enc_order = np.load("enc_order.npy")   # dtype: int
# Load FAISS index
index = faiss.read_index("jobs.index")
trans_df_final=pd.read_csv("trans_df_final.csv")

c:\Users\HP - Victus\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def predict_top_3_safe(query, k=3):
    qv = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(qv, k)
    results = []
    for rank, idx in enumerate(indices[0]):
        # safety checks:
        if idx < 0 or idx >= len(enc_order):
            continue
        row_idx = int(enc_order[idx])          # actual df row index
        job_info = trans_df_final.iloc[row_idx]
        results.append({
            "Rank": rank+1,
            "Job Title": job_info['Job Title'],
            "Sector": job_info['Sector'],
            "Role": job_info['Role'],
            "Average Salary (Aggregated)": job_info['Salary_avg_agg'],
            "Distance Score": float(distances[0][rank])
        })
    return results

In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
import uvicorn
nest_asyncio.apply()
app=FastAPI(title='Job recommendation')

class Query(BaseModel):
    text :str
    k: int =3


@app.post('/embed')
def embed(query : Query):
    emb = model.encode([query.text])[0].to_list()
    return {"embedding":emb}




@app.post('/recommend')
def recommend(query:Query):
    # Encode query
    results = predict_top_3_safe(query.text,query.k)

    return {"results": results}

uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     41.46.64.170:0 - "POST /recommend HTTP/1.1" 422 Unprocessable Entity
INFO:     41.46.64.170:0 - "POST /recommend HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3396]
